# Разметка lenta-ru с помощью deeppavlov

In [1]:
import pandas as pd
from tqdm import tqdm
from corus import load_lenta
from deeppavlov import build_model
from transformers import AutoTokenizer

random_state = 42

In [2]:
# !curl -L https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -o data/lenta-ru-news.csv.gz

In [3]:
ner_model = build_model('ner_collection3_bert', download=True, install=True)

2025-04-16 12:24:21.705 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_bert_coll3_torch.tar.gz download because of matching hashes
c:\Users\verai\projects\AITH_dl_nlp\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenC

In [4]:
records = load_lenta('data/lenta-ru-news.csv.gz')
data = pd.DataFrame(records)
data.columns = ['url', 'title', 'text', 'topic', 'tags', 'date']
data = data.sample(n=10000, random_state=random_state)
data = data[['text']].reset_index(drop=True)
data.head()

,text
0,Египетский перевозчик EgyptAir сообщил о возмо...
1,Глава Красногорского района Московской области...
2,Депутат Виталий Милонов внес в Госдуму законоп...
3,Верховный суд Индии разрешил женщинам в фертил...
4,Россиянам не стоит бояться роста цен на хлеб —...


In [5]:
print(ner_model([data["text"][0]]))

[[['Египетский', 'перевозчик', 'EgyptAir', 'сообщил', 'о', 'возможном', 'повышении', 'стоимости', 'билетов', 'на', 'свои', 'международные', 'рейсы', 'из', '‑', 'за', 'девальвации', 'национальной', 'валюты.', 'Такое', 'заявление', 'сделал', 'генеральный', 'директор', 'перевозчика', 'Шериф', 'Фатхи', ',', 'его', 'слова', 'приводит', '«', 'Рамблер.Путешествия', '»', '.', 'Фатхи', 'заверил', ',', 'что', 'пока', 'компания', 'не', 'приняла', 'решение', ',', 'но', 'признал', ',', 'что', '«', 'рост', 'неизбежен', '»', '.', 'При', 'этом', ',', 'пока', 'тарифы', 'на', 'перевозку', 'обсуждаются', ',', 'на', 'сайте', 'авиакомпании', 'билеты', 'уже', 'подорожали.', 'В', 'руководстве', 'авиакомпании', 'это', 'объяснили', 'изменением', 'в', 'налогообложении.', 'Предполагается', ',', 'что', 'повышение', 'цен', 'коснется', 'только', 'международных', 'перелетов', ',', 'стоимость', 'внутренних', 'рейсов', 'останется', 'прежней.', 'Отмечается', ',', 'что', 'центральный', 'банк', 'Египта', 'девальвировал',

In [6]:
def select_texts_by_token_count(text_list, pretrained_model, token_limit):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    result_texts = []
    for sentence in text_list:
        tokenized = tokenizer.tokenize(sentence)
        if len(tokenized) <= token_limit:
            result_texts.append(sentence)
    return result_texts

In [7]:
filtered_texts = select_texts_by_token_count(data['text'], "DeepPavlov/rubert-base-cased", 450)

c:\Users\verai\projects\AITH_dl_nlp\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
synthetic_annots = [ner_model([text]) for text in tqdm(filtered_texts)]
print(synthetic_annots[0])

100%|██████████| 9713/9713 [04:53<00:00, 33.04it/s]

[[['Египетский', 'перевозчик', 'EgyptAir', 'сообщил', 'о', 'возможном', 'повышении', 'стоимости', 'билетов', 'на', 'свои', 'международные', 'рейсы', 'из', '‑', 'за', 'девальвации', 'национальной', 'валюты.', 'Такое', 'заявление', 'сделал', 'генеральный', 'директор', 'перевозчика', 'Шериф', 'Фатхи', ',', 'его', 'слова', 'приводит', '«', 'Рамблер.Путешествия', '»', '.', 'Фатхи', 'заверил', ',', 'что', 'пока', 'компания', 'не', 'приняла', 'решение', ',', 'но', 'признал', ',', 'что', '«', 'рост', 'неизбежен', '»', '.', 'При', 'этом', ',', 'пока', 'тарифы', 'на', 'перевозку', 'обсуждаются', ',', 'на', 'сайте', 'авиакомпании', 'билеты', 'уже', 'подорожали.', 'В', 'руководстве', 'авиакомпании', 'это', 'объяснили', 'изменением', 'в', 'налогообложении.', 'Предполагается', ',', 'что', 'повышение', 'цен', 'коснется', 'только', 'международных', 'перелетов', ',', 'стоимость', 'внутренних', 'рейсов', 'останется', 'прежней.', 'Отмечается', ',', 'что', 'центральный', 'банк', 'Египта', 'девальвировал',

In [9]:
tag_map = {
    'S-LOC': 'B-LOC',
    'E-LOC': 'I-LOC',
    'S-ORG': 'B-ORG',
    'E-ORG': 'I-ORG',
    'S-PER': 'B-PER',
    'E-PER': 'I-PER'
}

adjusted_annots = [[annot[0], [[tag_map.get(tag, tag) for tag in annot[1][0]]]] for annot in synthetic_annots]

In [10]:
# Уникальные теги с учётом замен
unique_tags = {tag for annot in adjusted_annots for tag in annot[1][0]}
print("Уникальные BIO-теги после замены:", unique_tags)

Уникальные BIO-теги после замены: {'I-LOC', 'I-ORG', 'B-ORG', 'B-PER', 'I-PER', 'B-LOC', 'O'}


In [11]:
# Создаем DataFrame с текстами и обновлённой синтетической разметкой
df_annots = pd.DataFrame({"text": filtered_texts,
                          "annotation": adjusted_annots})

In [12]:
# Сохраняем DataFrame в формате Parquet
df_annots.to_parquet('data/synthetic_annotations.parquet', index=False)